<a name="2"></a>
## 2 -  Problem Statement

Suppose you are starting a company that grows and sells wild mushrooms. 
- Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes
- You have some existing data that you can use for this task. 

Can you use the data to help you identify which mushrooms can be sold safely? 

Note: The dataset used is for illustrative purposes only. It is not meant to be a guide on identifying edible mushrooms.



<a name="3"></a>
## 3 - Dataset

You will start by loading the dataset for this task. The dataset you have collected is as follows:

| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |


-  You have 10 examples of mushrooms. For each example, you have
    - Three features
        - Cap Color (`Brown` or `Red`),
        - Stalk Shape (`Tapering` or `Enlarging`), and
        - Solitary (`Yes` or `No`)
    - Label
        - Edible (`1` indicating yes or `0` indicating poisonous)

<a name="3.1"></a>
### 3.1 One hot encoded dataset
For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features)

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

Therefore,
- `X_train` contains three features for each example 
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible 
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

In [1]:
#one-hot encoding to upper data
import numpy as np
x_train=np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train=np.array([1,1,0,0,1,0,0,1,1,0])
print('x_train shape is:',x_train.shape)
print('y_train shape is:',y_train.shape)
print('number of training data(m) is:',len(x_train))

x_train shape is: (10, 3)
y_train shape is: (10,)
number of training data(m) is: 10


In [2]:
# calculate entropy of node
def compute_entropy(y):
    entropy=0.
    if len(y)>0:
        p1=len(y[y==1])/len(y)
        if p1!=0 and p1!=1:
            entropy=-p1*np.log2(p1)-(1-p1)*np.log2(1-p1)
        else:
            entropy=0.
    return entropy

In [4]:
print("entropy at root node is: ",compute_entropy(y_train))

entropy at root node is:  1.0


In [11]:
#spilt dataset
# Next, you'll write a helper function called `split_dataset` that takes in the data at a node and 
# a feature to split on and splits it into left and right branches.
# Later in the lab, you'll implement code to calculate how good the split is.


def split_dataset(x,node_indices,feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        x (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (ndarray):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (ndarray): Indices with feature value == 1
        right_indices (ndarray): Indices with feature value == 0
    """
    left_indices=[]
    right_indices=[]
    for i in node_indices:
        if x[i][feature]==1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

In [12]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(x_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)


Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [15]:
#calculate information gain
def compute_information_gain(x, y, node_indices, feature):
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        x (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    #spilt dataset
    left_indices, right_indices=split_dataset(x, node_indices, feature)
    #useful variables
    x_node,y_node=x[node_indices],y[node_indices]
    x_left,y_left=x[left_indices],y[left_indices]
    x_right,y_right=x[right_indices],y[right_indices]
    
    information_gain=0
    node_entropy=compute_entropy(y_node)
    left_entropy=compute_entropy(y_left) 
    right_entropy=compute_entropy(y_right) 
    #weights
    w_left=len(x_left)/len(x_node)
    w_right=len(x_right)/len(x_node)
    weight_entropy=w_left*left_entropy+w_right*right_entropy
    information_gain=node_entropy-weight_entropy
    return information_gain
    

In [18]:
#get best split
def get_best_split(x,y,node_indices):
    m_features=x.shape[1]
    best_feature=-1
    best_gain=0
    for feature in range(m_features):
        info_gain=compute_information_gain(x,y,node_indices,feature)
        if info_gain>best_gain:
            best_gain=info_gain
            best_feature=feature
    return best_feature

In [19]:
best_feature = get_best_split(x_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [20]:
#building the tree
#使用递归，遇到最大深度时停止
tree=[]
def build_tree_recursive(x,y,node_indices,branch_name,max_depth,current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """
    # Maximum depth reached - stop splitting
    if current_depth==max_depth:
        formatting=" "*current_depth+"-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature=get_best_split(x,y,node_indices)
    tree.append((current_depth,branch_name,best_feature,node_indices))
    
    formatting="-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
        # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(x, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(x, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(x, y, right_indices, "Right", max_depth, current_depth+1)

    
    
    
         


In [21]:
build_tree_recursive(x_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
